In [3]:
import pandas as pd
import json
import requests
import time
import re
import os
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
#df_movie_actualise avec les colonnes suivantes : adult	backdrop_path	genre_ids	id	original_language	original_title	overview	popularity	poster_path	release_date	title	video	vote_average	vote_count

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603


In [5]:
# On crée un dict avec les genres : 
df_genre = pd.read_parquet("genre.parquet")
df_genre = df_genre.set_index('id')
genre_string = df_genre.to_json()
genre_dict = json.loads(genre_string)
genre_dict['name']

{'28': 'Action',
 '12': 'Adventure',
 '16': 'Animation',
 '35': 'Comedy',
 '80': 'Crime',
 '99': 'Documentary',
 '18': 'Drama',
 '10751': 'Family',
 '14': 'Fantasy',
 '36': 'History',
 '27': 'Horror',
 '10402': 'Music',
 '9648': 'Mystery',
 '10749': 'Romance',
 '878': 'Science Fiction',
 '10770': 'TV Movie',
 '53': 'Thriller',
 '10752': 'War',
 '37': 'Western'}

In [6]:
# replacer les id des genres dans le df_movie par la liste des genres
def str_genre(list_genre: list, genre_dict: dict)-> str:
    empty_list = []
    for element in list_genre : 
        element = str(element)
        if element in genre_dict: 
            empty_list.append(genre_dict[element])
    return empty_list

In [7]:
# on applique la fonction et on crée une nouvelle colonne 'genre'
df_movie_actualise['genre'] = df_movie_actualise['genre_ids'].apply(lambda x: str_genre(x, genre_dict['name']))
df_movie_actualise.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685,"[Family, Fantasy, Animation, Romance]"
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655,[Drama]
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603,"[Drama, History]"


In [8]:
# la fonction pour détécter tous les titres qui ne sont pas en lettres latines
def isascii(title: str) ->str: 
    car_special = ['à', 'â', 'ä', 'é', 'è', 'ê', 'ë', 'î', 'ï', 'ô', 'ö', 'ù', 'û', 'ü', 'ç']
    if title.isascii() or any(char in title for char in car_special):
        return title
    else:
        return 0

In [9]:
# on applique la fonction et on crée une nouvelle colonne 'isascii'
df_movie_actualise['isascii'] = df_movie_actualise['title'].apply(isascii)
df_movie_actualise.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre,isascii
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685,"[Family, Fantasy, Animation, Romance]",Cendrillon
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655,[Drama],12 Hommes en colère
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603,"[Drama, History]",Les Dix Commandements


In [10]:
# on supprime tout ce qui n'est pas ascii et on drop la colonne 'isascii'
df_movies_updated = df_movie_actualise[df_movie_actualise["isascii"] != 0]
df_movies_updated.drop(columns=['isascii'], inplace=True)
df_movies_updated.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685,"[Family, Fantasy, Animation, Romance]"
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655,[Drama]
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603,"[Drama, History]"


In [23]:
# POur récupérer df_cast
df_cast = pd.DataFrame()
i = 0
for movie_id in df_movie_actualise["id"]:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?"
    i += 1
    headers = {
        "accept": "application/json",
        "Authorization": "tmdb_key"
    }
    r = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(r.text), record_path="cast")
    df_new["movie_id"] = movie_id
    if df_cast.empty:
        df_cast = df_new
    else:
        df_cast = pd.concat([df_cast, df_new], ignore_index=True)
    if i % 40:
        time.sleep(1)
    print(f"\rProgress: {i} on 5772", end='', flush=True)

df_cast


Progress: 3 on 5772

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,1,68652,Acting,Ilene Woods,Ilene Woods,8.444,/ApIXw423GZoMnwYUmkqEq5awpng.jpg,1,Cinderella (voice),52fe44149251416c750282ff,0,11224
1,False,1,67289,Acting,Eleanor Audley,Eleanor Audley,7.598,/rppllmBSCZhkhCWoomVvbB1pB4T.jpg,2,Lady Tremaine (voice),52fe44149251416c75028303,1,11224
2,False,1,67290,Acting,Verna Felton,Verna Felton,5.372,/xJ7mohLuufsNn6dkvzjKi5nlaFn.jpg,3,Fairy Godmother (voice),52fe44149251416c75028307,2,11224
3,False,1,68653,Acting,Claire Du Brey,Claire Du Brey,4.683,/e8WDoA2IhvjkFMZ2OEsr7PinZEe.jpg,52,(credit only) (voice),585c3b03c3a3687fa400009f,3,11224
4,False,1,179371,Acting,Rhoda Williams,Rhoda Williams,6.270,/qKNBo1p359fWPko78DzRHXwVUZt.jpg,21,Drizella (voice),52fe44149251416c7502836b,4,11224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,False,2,34168,Acting,Frank Hagney,Frank Hagney,3.804,/eUun6Blqa6hEBrlscgz9qnH6Ly0.jpg,231,Hebrew at Golden Calf (uncredited),647dad5e93828e00a765722a,108,6844
150,False,2,151727,Acting,Paul Wexler,Paul Wexler,2.003,/9g7fq97hqkHeQ1lbU6CqZJsaeOp.jpg,233,Hebrew at Golden Calf,64975920eb79c200ad7162c3,109,6844
151,False,2,1096850,Acting,Charles Stevens,Charles Stevens,1.835,/5mCi2fkG5pn68RXaNSDa6eVouPm.jpg,234,Slave (uncredited),64be7d1bb865eb01399d51f1,110,6844
152,False,2,544425,Acting,Steve Mitchell,Steve Mitchell,1.204,/aMs3QkRmgrHOsZHqgfj7QCEYv1q.jpg,235,Slave (uncredited),64c18a2cdb4ed600aeb0320b,111,6844


In [28]:
df_cast.to_parquet("df_cast.parquet")
df_actor = pd.read_parquet("df_cast.parquet")
df_actor.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,1,68652,Acting,Ilene Woods,Ilene Woods,8.444,/ApIXw423GZoMnwYUmkqEq5awpng.jpg,1,Cinderella (voice),52fe44149251416c750282ff,0,11224
1,False,1,67289,Acting,Eleanor Audley,Eleanor Audley,7.598,/rppllmBSCZhkhCWoomVvbB1pB4T.jpg,2,Lady Tremaine (voice),52fe44149251416c75028303,1,11224
2,False,1,67290,Acting,Verna Felton,Verna Felton,5.372,/xJ7mohLuufsNn6dkvzjKi5nlaFn.jpg,3,Fairy Godmother (voice),52fe44149251416c75028307,2,11224
3,False,1,68653,Acting,Claire Du Brey,Claire Du Brey,4.683,/e8WDoA2IhvjkFMZ2OEsr7PinZEe.jpg,52,(credit only) (voice),585c3b03c3a3687fa400009f,3,11224
4,False,1,179371,Acting,Rhoda Williams,Rhoda Williams,6.270,/qKNBo1p359fWPko78DzRHXwVUZt.jpg,21,Drizella (voice),52fe44149251416c7502836b,4,11224


In [30]:
# dans df_actor ne garder que les ordres 1-4 et que les departement Acting
df_actor = df_actor[df_actor['order'] <= 4]

df_actor = df_actor[df_actor['known_for_department'] =='Acting']
df_actor.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,1,68652,Acting,Ilene Woods,Ilene Woods,8.444,/ApIXw423GZoMnwYUmkqEq5awpng.jpg,1,Cinderella (voice),52fe44149251416c750282ff,0,11224
1,False,1,67289,Acting,Eleanor Audley,Eleanor Audley,7.598,/rppllmBSCZhkhCWoomVvbB1pB4T.jpg,2,Lady Tremaine (voice),52fe44149251416c75028303,1,11224
2,False,1,67290,Acting,Verna Felton,Verna Felton,5.372,/xJ7mohLuufsNn6dkvzjKi5nlaFn.jpg,3,Fairy Godmother (voice),52fe44149251416c75028307,2,11224
3,False,1,68653,Acting,Claire Du Brey,Claire Du Brey,4.683,/e8WDoA2IhvjkFMZ2OEsr7PinZEe.jpg,52,(credit only) (voice),585c3b03c3a3687fa400009f,3,11224
4,False,1,179371,Acting,Rhoda Williams,Rhoda Williams,6.270,/qKNBo1p359fWPko78DzRHXwVUZt.jpg,21,Drizella (voice),52fe44149251416c7502836b,4,11224


In [24]:
# POur récupérer df_crew ()
df_crew = pd.DataFrame()
i = 0
for movie_id in df_movie_actualise["id"]:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?"
    i += 1
    headers = {
        "accept": "application/json",
        "Authorization": "tmdb_key"
    }
    r = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(r.text), record_path="crew")
    df_new["movie_id"] = movie_id
    if df_crew.empty:
        df_crew = df_new
    else:
        df_crew = pd.concat([df_crew, df_new], ignore_index=True)
    if i % 40:
        time.sleep(1)
    print(f"\rProgress: {i} on 5772", end='', flush=True)
df_crew.head()

Progress: 3 on 5772

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,movie_id
0,False,2,44217,Writing,Charles Perrault,Charles Perrault,6.546,/g6FQdZTTlI6r1nmS6ZgJ6w25yI6.jpg,52fe44149251416c7502831f,Writing,Novel,11224
1,False,2,2106,Production,Walt Disney,Walt Disney,11.041,/fnB3Yt4dPiVODHzfLwntISAvbhw.jpg,52fe44149251416c75028367,Production,Producer,11224
2,False,2,2107,Sound,Paul J. Smith,Paul J. Smith,2.000,None,52fe44149251416c75028355,Sound,Original Music Composer,11224
3,False,2,64869,Editing,Donald Halliday,Donald Halliday,2.779,None,52fe44149251416c75028361,Editing,Editor,11224
4,False,2,64868,Sound,Oliver Wallace,Oliver Wallace,1.439,None,52fe44149251416c7502835b,Sound,Original Music Composer,11224


In [31]:
# Dans df_crew ne garder que le job = Director
df_crew.to_parquet("df_crew.parquet")

df_director = pd.read_parquet("df_crew.parquet")
df_director = df_director[df_director['job'] == 'Director']
df_director.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,movie_id
5,False,2,64864,Visual Effects,Clyde Geronimi,Clyde Geronimi,10.792,/eAeqN6fPYuzKjGRSaR2jqhSeeMq.jpg,52fe44149251416c7502830d,Directing,Director,11224
6,False,2,11434,Directing,Wilfred Jackson,Wilfred Jackson,9.316,/rTszyDc1jFOQQrzSYgnxEVLxd8W.jpg,52fe44149251416c75028313,Directing,Director,11224
7,False,2,11429,Directing,Hamilton Luske,Hamilton Luske,7.353,/bEccjwuG6QGc4V0AtA87rexRHYo.jpg,52fe44149251416c75028319,Directing,Director,11224
87,False,2,39996,Directing,Sidney Lumet,Sidney Lumet,6.731,/Ac24Ya2bVvhW7sI5da041MeaeBT.jpg,5e8e3c2f62e86f00159ba2f3,Directing,Director,389
92,False,2,8636,Directing,Cecil B. DeMille,Cecil B. DeMille,4.396,/lxY61cx23rcCenlTvcwn8Cmpnrp.jpg,52fe4466c3a36847f8093ab1,Directing,Director,6844


In [ ]:
# PEOPLE : le lien pour récupérer le détail des réalisateurs :  
df_director_details = pd.DataFrame()
count  = 0
for id in df_director['id_director']:
    url = f"https://api.themoviedb.org/3/person/{id}?language=fr-FR'"

    headers = {
    "accept": "application/json",
    "Authorization": os.getenv("tmdb_key")}

    response = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(response.text))
    df_director_details = pd.concat([df_director_details, df_new], ignore_index=True)
    count += 1
    if count == 100:
        time.sleep(3)
        count = 0
df_director_details.to_parquet("df_director_details.parquet")

In [ ]:
# PEOPLE : le lien pour récupérer le détail des acteurs :
df_director_details = pd.DataFrame()
count  = 0
for id in df_director['id_director']:
    url = f"https://api.themoviedb.org/3/person/{id}?language=fr-FR'"

    headers = {
    "accept": "application/json",
    "Authorization": os.getenv("tmdb_key")}

    response = requests.get(url, headers=headers)
    df_new = pd.json_normalize(json.loads(response.text))
    df_director_details = pd.concat([df_director_details, df_new], ignore_index=True)
    count += 1
    if count == 100:
        time.sleep(3)
        count = 0
df_director_details.to_parquet("df_director_details.parquet")

merge le détail des directeurs avec la table directeurs 
merge le détail des acteurs avec la table acteurs 
merge la table des directeurs avec df_movie 

In [48]:
# Rajouter les colonnes acteur_1 - acteur_5 dans le dataframe des movies et directeurs: 

df_actors_final[f"name_actor_2"] = None

for movie_id in df_actors_final["movie_id"]:
    df_temp = df_actor[df_actor["movie_id"] == movie_id] 
    df_temp = df_temp.sort_values(by="order").reset_index()
    try :
        df_actors_final.loc[df_actors_final['movie_id'] == movie_id, f'name_actor_2'] = df_temp.loc[2, "name"]
    except:
        df_actors_final.loc[df_actors_final['movie_id'] == movie_id, f'name_actor_2'] = None
df_actors_final.sort_values(by=['movie_id'])

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre,name_actor_1,name_actor_2
1,False,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,[18],389,en,12 Angry Men,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,False,8.500,8655,[Drama],John Fiedler,Lee J. Cobb
2,False,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,"[18, 36]",6844,en,The Ten Commandments,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,False,7.756,1603,"[Drama, History]",Yul Brynner,Anne Baxter
0,False,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,"[10751, 14, 16, 10749]",11224,en,Cinderella,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,False,7.041,6685,"[Family, Fantasy, Animation, Romance]",Eleanor Audley,Verna Felton


In [ ]:
# On remplace les dates par les décennies dans le DF général (avec movies, directeurs et 5 colonnes d'acteurs) SI CA NE FONCTIONNE PAS, IL FAUT EXECUTER 2 FOIS

date_list = ['release_date', 'birthday_director', 'deathday_director']

for date in date_list:
    df_movie[date] = pd.to_datetime(df_movie[date])

df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date', 'birthday_director', 'deathday_director']].fillna(0)

for date in date_list:
    df_movie[date] = ((df_movie[date].dt.year // 10) * 10).astype(int)

df_movie

In [ ]:
# Boucle pour récupérer les noms des acteurs dans une liste : 
actor_list = []
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    for i in range(0, len(df_movie)): 
        acteur_name = df_movie.loc[i, acteur_name_col]
        if acteur_name not in actor_list: 
            actor_list.append(acteur_name)

In [ ]:
# Boucle pour récupérer la note moyenne des films par acteur -> le résultat est stocké dans le dict resultat 
resultat = {}

for acteur in actor_list: 
    index_list = []
    popularity_list = []
    for col in range(1, 6): 
        acteur_name_col = f"actor_name_{col}"
        
        index = df_movie[df_movie[acteur_name_col] == acteur].index
        for i in index : 
            index_list.append(i)

    for index in index_list:
        popularity = df_movie.loc[index, 'popularity_movie']
        popularity_list.append(popularity)
    
    try: 
        resultat[acteur] = float(round(sum(popularity_list) / len(popularity_list),2))
    except: 
        pass

In [ ]:
# On remplace les noms des acteurs par la note moyenne : On va peut-être garder les noms des acteurs...
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    for i in range(0, len(df_movie)): 
        df_movie.loc[i, acteur_name_col] = resultat.get(df_movie.loc[i, acteur_name_col])

df_movie.head(3)


In [ ]:
# On rénomme les colonnes 
df_movie_actualise = df_movie_actualise.rename(columns={'actor_name_1': 'actor_1_mean_movie', 
                         'actor_name_2': 'actor_2_mean_movie', 
                         'actor_name_3': 'actor_3_mean_movie',
                         'actor_name_4': 'actor_4_mean_movie', 
                         'actor_name_5': 'actor_5_mean_movie'
})

In [ ]:
# Remplir les valeurs nulles dans d'autres colonnes numériques
df_movie_actualise[['actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5',
          'actor_2_mean_movie', 'actor_3_mean_movie', 'actor_4_mean_movie', 'actor_5_mean_movie', 
          'actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4', 'actor_popularity_5']] = df_movie_actualise[['actor_gender_2', 'actor_gender_3', 
                                                                                                               'actor_gender_4', 'actor_gender_5',
                                                                                                               'actor_2_mean_movie', 'actor_3_mean_movie', 
                                                                                                               'actor_4_mean_movie', 'actor_5_mean_movie', 
                                                                                                               'actor_popularity_2', 'actor_popularity_3', 
                                                                                                               'actor_popularity_4', 'actor_popularity_5']].fillna(0)



# Remplir les valeurs nulls dans d'autres colonnes non-numériques : 
df_movie_actualise[['place_of_birth_director']] = df_movie_actualise[['place_of_birth_director']].fillna('aucun')

à la fin le fichier doit être identique au fichier "df_final_compile.parquet" sans les dates de naissance des acteurs
